In [1]:
import pandas as pd
import numpy as np
import re
df = pd.read_json('./sample_p4.txt', encoding='gb18030', lines=True)
df.dropna(axis=0, subset=['content'], inplace=True)
print('非空:', len(df))
df = df.iloc[np.bitwise_and(
    df['trial_round'].values != '其他', df['trial_round'].values != '刑罚变更'), :]
print('非其他，非刑法变更:', len(df))


def is_valid(t):
    if t.find('撤回上诉') != -1:
        return False
    if t.find('重新审判') != -1:
        return False
    if t.find("申诉理由不能成立") != -1:
        return False
    return True


df = df.iloc[df['content'].apply(is_valid).values, :]
print('非撤回上诉:', len(df))

fact = []


def get_money(t: str):
    ret = []

    m = re.findall(f"(?<=追缴赃款).*?元", t, re.M)
    ret.extend(m)

    m = re.findall(f"(?<=违法所得).*?元", t, re.M)
    ret.extend(m)

    m = re.findall(f"(?<=所得赃款).*?元", t, re.M)
    ret.extend(m)

    m = re.findall(f"(?<=收受的).*赃款依法予以没收", t, re.M)
    ret.extend(m)

    m = re.findall(f"(?<=收受的).*上缴", t, re.M)
    ret.extend(m)

    if len(ret) == 0:
        m = re.findall(f"(?<=共计).*元", t, re.M)
        ret.extend(m)

    return ret


for idx, row in df.iterrows():
    text = row['content']
    bg = text.find("经审理查明")
    ed = text.find("本院认为")
    text = text[bg+6:ed]
    fact.append(text)

df.insert(2, 'fact', fact)

money = []
cnt = 1

for idx, row in df.iterrows():

    m = get_money(row['content'])

    # if(len(m) == 0):
    #     cnt += 1
    #     print(row['content'])

    money.append(m)

print(money)


非空: 951
非其他，非刑法变更: 624
非撤回上诉: 564
[['人民币四万五千元'], ['款17251元'], ['人民币12万元'], ['三万元赃款依法予以没收', '三万元赃款依法予以没收，上缴'], ['人民币117000元', '人民31000元', '人民币117000元', '人民币19000元，并答应有机会提拔韩某甲，但至今没有实现。2014年8月间，芗城二实小校长黄某乙被纪检部门查处，其担心以前收受他人财物的事情被发现，于2015年3月，退给韩某甲人民币10000元', '人民币13000元', '人民币10000元', '人民币7000元', '人民币5000元', '人民币8000元', '人民币117000元'], ['人民币186.8198万元'], ['人民币四万元'], ['现金人民币62000元', '人民币44000元', '人民币13000元', '人民币5000元'], ['27万元人民币，其中，被告人杨汉英收受17万元，被告人万昌平收受10万元'], ['60000元', '8000元', '8000元', '5000元'], ['人民币二十八万元', '人民币十五万五千元'], ['，依法予以没收，上缴国库，剩余6714.4元'], ['人民币七万元', '（未退还部分）人民币六万元'], ['人民币103000元', '人民币80000元', '人民币23000元', '人民币40万元', '人民币103000元', '人民币103000元', '人民币80000元', '人民币20000元', '人民币20000元', '人民币20000元', '人民币20000元', '人民币23000元', '人民币40万元', '人民币8万元。第一次是在2009年中秋节前的一个星期，黄某约我去他的办公室喝茶。当时只有我和他两个人，喝茶的过程中，黄某问我今年工程款的资金安排表出来没有，分配完工程款后镇政府的财政结余还多不多，如果还有的话，他会找镇长、书记活动想下办法，看能不能尽快拿齐工程款。我就告诉他今年的工程款分配计划出来了，但还没到我手上，还在镇长手里，我还告诉他财政还有支付工程款的机动资金以备使用的。然后黄某就从他的包里拿出一个信封给我说“过节了，表示一下，你自己买点东西过节。”我收下信封回到办公室之后打开信封一看